# Prepare the base

#### Import data and python libraries

In [597]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [598]:
base = pd.read_excel('/content/sample_data/dados_asset_bills.xlsx', sheet_name = 'Planilha1')

#### Transform data

In [599]:
base_test = base

In [600]:
# Calculate the aging of due_date
base_test['aging_due_date'] = (base_test['new_due_date'] -  base_test['due_date']).dt.days
# Select useful columns
base_test = base_test[['NOME_ENDOSSER', 'kind', 'state', 'aging_due_date', 'update_reason_kind']]
# Filling the NaN values from column state for finished duplicates
base_test['update_reason_kind'] =  base_test['update_reason_kind'].fillna('not canceled')
# One hot encoding for kind
base_test = pd.get_dummies(base_test, columns = ['kind'],dtype=int)


<ipython-input-600-9db6db7bce51>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_test['update_reason_kind'] =  base_test['update_reason_kind'].fillna('not canceled')


In [601]:
# One hot encoding to state
# Separate the column for one hot
categorical_attributes = ['state']
categorical_columns = base_test[categorical_attributes]
# instance and train OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(categorical_columns)
# Codify the categorical columns and transform in a dataframe
encoded = encoder.transform(categorical_columns).toarray()
enc_train = pd.DataFrame(data = encoded, columns = encoder.categories_)
# Concat with the original dataframe and drop original column
base_test = pd.concat([base_test,enc_train],axis=1)
base_test.drop(categorical_attributes, axis=1, inplace=True)

In [602]:
# One hot encoding for update_reason_kind
# Separate the column for one hot
categorical_attributes = ['update_reason_kind']
categorical_columns = base_test[categorical_attributes]
# instance and train OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(categorical_columns)
# Codify the categorical columns and transform in a dataframe
encoded = encoder.transform(categorical_columns).toarray()
enc_train = pd.DataFrame(data = encoded, columns = encoder.categories_)
# Concat with the original dataframe and drop original column
base_test = pd.concat([base_test,enc_train],axis=1)
base_test.drop(categorical_attributes, axis=1, inplace=True)


In [603]:
# Rename columns
base_test.columns

Index([       'NOME_ENDOSSER',       'aging_due_date',           'kind_goods',
              'kind_services',            ('active',),          ('canceled',),
                ('finished',),      ('not canceled',), ('operational_error',),
                  ('others',),          ('reversal',)],
      dtype='object')

In [604]:
base_test = base_test.rename(columns={ ('active',): "active", ('canceled',): "canceled", ('finished',): 'finished', ('not canceled',): 'not_canceled', ('operational_error',): 'operational_error', ('others',): 'others', ('reversal',): 'reversal'})

In [605]:
base_test.isna().sum()

,0
NOME_ENDOSSER,0
aging_due_date,0
kind_goods,0
kind_services,0
active,0
canceled,0
finished,0
not_canceled,0
operational_error,0
others,0


In [606]:
base_test.head()

,NOME_ENDOSSER,aging_due_date,kind_goods,kind_services,active,canceled,finished,not_canceled,operational_error,others,reversal
0,Renault do Brasil S/A,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,KABUM COMERCIO ELETRONICO S.A,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,KABUM COMERCIO ELETRONICO S.A,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,KABUM COMERCIO ELETRONICO S.A,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,KABUM COMERCIO ELETRONICO S.A,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [607]:
function_dictionary = {'aging_due_date':'mean','kind_goods':'sum','kind_services':'sum', 'active':'sum', 'canceled':'sum', 'finished': 'sum', 'not_canceled': 'sum', 'operational_error': 'sum',	'others': 'sum',	'reversal': 'sum'}
base_crazy = base_test
base_test = base_test.groupby("NOME_ENDOSSER").aggregate(function_dictionary).reset_index(0)

In [608]:
base_nfe = base

In [609]:
# base_nfe.groupby("NOME_ENDOSSER").agg({"nfe_number": pd.Series.nunique})
# base_nfe.groupby(['NOME_ENDOSSER', 'nfe_number']).agg({'nfe_number': 'count'})
base_nfe = base_nfe.groupby(["NOME_ENDOSSER", "nfe_number"]).agg({"id": pd.Series.nunique}).reset_index(0)

In [610]:
base_nfe = base_nfe.reset_index(0)

In [611]:
base_nfe = base_nfe.groupby("NOME_ENDOSSER").agg({"id": 'median'}).reset_index(0)

In [612]:
base_test = base_test.set_index('NOME_ENDOSSER').join(base_nfe.set_index('NOME_ENDOSSER')).reset_index(0)

In [613]:
base_test = base_test.rename(columns={ 'id': "installment median"})

In [614]:
base_test.fillna(base_test['installment median'].median(), inplace=True)

In [615]:
base_test[base_test['active'] != 0].sort_values(by=['active'], ascending=True).head(10)

,NOME_ENDOSSER,aging_due_date,kind_goods,kind_services,active,canceled,finished,not_canceled,operational_error,others,reversal,installment median
0,2001-08-01 00:00:00,0.0,0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
433,Oliver Print Adesivos Ltda,0.0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
434,Original,0.0,0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
448,PG1,0.0,0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
456,PORTO SEGURO COMPANHIA DE SEGUROS G ERAIS,0.0,2,0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0
146,Casas Bahia,0.0,0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
142,COPA INDUSTRIAL LTDA,0.0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
140,COOPER SUP COOPER PRODS.AGR.IMP E EXP F3,0.0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
136,CONDUSCABOS BRASIL INDUSTRIA COMERCIO DE,0.0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
129,COMERCIO DE CEREAIS DELLA LTDA,0.0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [616]:
base_test['total'] = base_test['active'] + base_test['canceled'] + base_test['finished']

In [617]:
base_test.sort_values(by=['active'], ascending = False).head(10)

,NOME_ENDOSSER,aging_due_date,kind_goods,kind_services,active,canceled,finished,not_canceled,operational_error,others,reversal,installment median,total
317,KABUM COMERCIO ELETRONICO S.A,0.0,151521,0,109148.0,42370.0,3.0,109151.0,9091.0,357.0,32922.0,1.0,151521.0
45,ANTONIA SELMA AGUIAR PARENTE,0.0,1375,0,1375.0,0.0,0.0,1375.0,0.0,0.0,0.0,1.0,1375.0
352,MAIS DISTRIBUIDORA LTDA,0.0,1285,0,1259.0,0.0,26.0,1285.0,0.0,0.0,0.0,2.0,1285.0
267,IMBRALIT INDUSTRIA E COMERCIO DE ARTEFATOS E F...,0.0,1037,0,1037.0,0.0,0.0,1037.0,0.0,0.0,0.0,1.0,1037.0
77,BRASCOPPER CBC BRASILEIRA DE CONDUTORES LTDA ...,0.0,808,0,790.0,0.0,18.0,808.0,0.0,0.0,0.0,4.0,808.0
321,Kabum S/A,0.0,662,0,662.0,0.0,0.0,662.0,0.0,0.0,0.0,1.0,662.0
526,SOBRAL HPC DISTRIBUIDORA LTDA,0.0,636,0,636.0,0.0,0.0,636.0,0.0,0.0,0.0,1.0,636.0
232,GERALDO ARAUJO TECIDOS LTDA,0.0,513,0,513.0,0.0,0.0,513.0,0.0,0.0,0.0,4.0,513.0
372,MEGA COMPONENTES LTDA (SEDE RIO DE JANEIRO),0.0,512,0,500.0,0.0,12.0,512.0,0.0,0.0,0.0,3.0,512.0
184,EMPRESA NACIONAL DE MATERIAIS DE CONSTRUCAO LTDA,0.0,479,0,479.0,0.0,0.0,479.0,0.0,0.0,0.0,2.0,479.0


In [618]:
base_test

,NOME_ENDOSSER,aging_due_date,kind_goods,kind_services,active,canceled,finished,not_canceled,operational_error,others,reversal,installment median,total
0,2001-08-01 00:00:00,0.000000,0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,LEAO NORDESTE INDUSTRIA E COMERCIO DE PLASTIC...,0.000000,50,0,50.0,0.0,0.0,50.0,0.0,0.0,0.0,1.0,50.0
2,RAIADROGASIL S/A,-0.181818,11,0,7.0,2.0,2.0,9.0,1.0,1.0,0.0,11.0,11.0
3,Shopping São Paulo,0.000000,0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
4,027 SMP - IND. COMERCIO MOVEIS LTDA,0.000000,12,0,12.0,0.0,0.0,12.0,0.0,0.0,0.0,4.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,testeFa,0.000000,1,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
605,testeFabricio,0.000000,2,0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0
606,testeFalha500,0.000000,0,30,30.0,0.0,0.0,30.0,0.0,0.0,0.0,1.0,30.0
607,ultragaz,0.000000,0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0


In [619]:
# create percentage of not canceled
base_test['percent_not_canceled'] = base_test['not_canceled'] / base_test['total']

#### Create scoring

In [620]:
base_test['score_finished'] = np.where(
    base_test["finished"] <= 5,(1000*base_test["finished"])/5,
    np.where(
        (base_test["finished"] > 5) & (base_test["finished"] <= 18),(3000*base_test["finished"])/18,
    np.where(
        (base_test["finished"] > 18) & (base_test["finished"] <= 30), (4999*base_test["finished"])/30,
        5000
    )
        )
    )

In [621]:
base_test['score_active'] = np.where(
    base_test["active"] <= 40,(600*base_test["active"])/40,
    np.where(
        (base_test["active"] > 40) & (base_test["active"] <= 199),(1800*base_test["active"])/199,
    np.where(
        (base_test["active"] > 199) & (base_test["active"] <= 484), (2999*base_test["active"])/484,
        3000
    )
        )
    )

In [622]:
base_test['score_total'] = np.where(
    base_test["total"] <= 40,(400*base_test["total"])/40,
    np.where(
        (base_test["total"] > 40) & (base_test["total"] <= 100),(1200*base_test["total"])/100,
    np.where(
        (base_test["total"] > 100) & (base_test["total"] <= 200), (1999*base_test["total"])/200,
        2000
    )
        )
    )

In [623]:
base_test['score_canceled'] = np.where(
    base_test["canceled"] <= 13,(200*base_test["canceled"])/13,
    np.where(
        (base_test["canceled"] > 13) & (base_test["canceled"] <= 35),(600*base_test["canceled"])/35,
    np.where(
        (base_test["canceled"] > 35) & (base_test["canceled"] <= 65), (999*base_test["canceled"])/65,
        1000
    )
        )
    )

In [624]:
base_test['score'] = base_test['score_finished'] + base_test['score_active'] + base_test['score_total'] - base_test['score_canceled']

In [625]:
base_test.sort_values(by=['total'], ascending=False).head(10)

,NOME_ENDOSSER,aging_due_date,kind_goods,kind_services,active,canceled,finished,not_canceled,operational_error,others,reversal,installment median,total,percent_not_canceled,score_finished,score_active,score_total,score_canceled,score
317,KABUM COMERCIO ELETRONICO S.A,0.0,151521,0,109148.0,42370.0,3.0,109151.0,9091.0,357.0,32922.0,1.0,151521.0,0.720369,600.000000,3000.000000,2000.0,1000.0,4600.000000
45,ANTONIA SELMA AGUIAR PARENTE,0.0,1375,0,1375.0,0.0,0.0,1375.0,0.0,0.0,0.0,1.0,1375.0,1.000000,0.000000,3000.000000,2000.0,0.0,5000.000000
352,MAIS DISTRIBUIDORA LTDA,0.0,1285,0,1259.0,0.0,26.0,1285.0,0.0,0.0,0.0,2.0,1285.0,1.000000,4332.466667,3000.000000,2000.0,0.0,9332.466667
267,IMBRALIT INDUSTRIA E COMERCIO DE ARTEFATOS E F...,0.0,1037,0,1037.0,0.0,0.0,1037.0,0.0,0.0,0.0,1.0,1037.0,1.000000,0.000000,3000.000000,2000.0,0.0,5000.000000
77,BRASCOPPER CBC BRASILEIRA DE CONDUTORES LTDA ...,0.0,808,0,790.0,0.0,18.0,808.0,0.0,0.0,0.0,4.0,808.0,1.000000,3000.000000,3000.000000,2000.0,0.0,8000.000000
321,Kabum S/A,0.0,662,0,662.0,0.0,0.0,662.0,0.0,0.0,0.0,1.0,662.0,1.000000,0.000000,3000.000000,2000.0,0.0,5000.000000
526,SOBRAL HPC DISTRIBUIDORA LTDA,0.0,636,0,636.0,0.0,0.0,636.0,0.0,0.0,0.0,1.0,636.0,1.000000,0.000000,3000.000000,2000.0,0.0,5000.000000
255,HERA METAIS IND COM PERF DE ALUM LTDA,0.0,539,0,471.0,0.0,68.0,539.0,0.0,0.0,0.0,2.0,539.0,1.000000,5000.000000,2918.448347,2000.0,0.0,9918.448347
232,GERALDO ARAUJO TECIDOS LTDA,0.0,513,0,513.0,0.0,0.0,513.0,0.0,0.0,0.0,4.0,513.0,1.000000,0.000000,3000.000000,2000.0,0.0,5000.000000
372,MEGA COMPONENTES LTDA (SEDE RIO DE JANEIRO),0.0,512,0,500.0,0.0,12.0,512.0,0.0,0.0,0.0,3.0,512.0,1.000000,2000.000000,3000.000000,2000.0,0.0,7000.000000


# IA - MODEL Nº2


In [626]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score # Utilizado para separar dados de treino e teste
from sklearn.preprocessing import StandardScaler # Utilizado para fazer a normalização dos dados
from sklearn.preprocessing import MinMaxScaler # Utilizado para fazer a normalização dos dados
from sklearn.preprocessing import LabelEncoder # Utilizado para fazer o OneHotEncoding
from sklearn.linear_model import LinearRegression # Algoritmo de Regressão Linear
from sklearn.metrics import r2_score, accuracy_score,confusion_matrix, ConfusionMatrixDisplay # Métricas de avaliação do models
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn import tree

In [627]:
base_test.describe()

,aging_due_date,kind_goods,kind_services,active,canceled,finished,not_canceled,operational_error,others,reversal,installment median,total,percent_not_canceled,score_finished,score_active,score_total,score_canceled,score
count,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000
mean,0.986889,296.154351,1.446634,225.420361,71.408867,0.771757,226.192118,15.963875,1.315271,54.129721,1.878489,297.600985,0.932982,124.625287,365.119058,375.336182,26.322785,838.757743
std,15.422531,6139.222428,14.242179,4422.659613,1716.876230,3.916328,4422.784171,368.430056,15.726563,1334.064068,1.612813,6139.171723,0.217433,512.374054,566.325415,566.509130,127.852359,1343.554071
min,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,10.000000,0.000000,-250.000000
25%,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,3.000000,1.000000,0.000000,30.000000,30.000000,0.000000,75.000000
50%,0.000000,10.000000,0.000000,9.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,1.000000,10.000000,1.000000,0.000000,135.000000,100.000000,0.000000,250.000000
75%,0.000000,38.000000,0.000000,36.000000,0.000000,0.000000,37.000000,0.000000,0.000000,0.000000,2.500000,40.000000,1.000000,0.000000,452.261307,400.000000,0.000000,947.035176
max,366.000000,151521.000000,312.000000,109148.000000,42370.000000,68.000000,109151.000000,9091.000000,357.000000,32922.000000,18.500000,151521.000000,1.000000,5000.000000,3000.000000,2000.000000,1000.000000,9918.448347


In [628]:
# Separate data for use
base_model = base_test[['aging_due_date',	'kind_goods',	'kind_services',	'active',	'canceled',	'finished',	'not_canceled',	'installment median',	'total','score']]
base_model = base_model[base_model['total'] <= 1000]

In [629]:
base_model.describe()

,aging_due_date,kind_goods,kind_services,active,canceled,finished,not_canceled,installment median,total,score
count,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000
mean,0.993414,41.553719,1.456198,40.433058,1.847934,0.728926,41.161983,1.882645,43.009917,804.745453
std,15.473305,88.119778,14.288773,86.794064,10.466599,3.791229,87.979190,1.616947,88.775555,1271.114278
min,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-250.000000
25%,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,2.000000,1.000000,3.000000,75.000000
50%,0.000000,10.000000,0.000000,9.000000,0.000000,0.000000,9.000000,1.000000,10.000000,250.000000
75%,0.000000,37.000000,0.000000,34.000000,0.000000,0.000000,36.000000,2.500000,39.000000,925.000000
max,366.000000,808.000000,312.000000,790.000000,125.000000,68.000000,808.000000,18.500000,808.000000,9918.448347


In [630]:
X = base_model.loc[ : , base_model.columns != 'score']
y = base_model['score']

In [631]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [632]:
# Normalization
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [633]:
# Treina o modelo
model = LinearRegression()
model = model.fit(X_train, y_train)

In [634]:
# Accuracy
r2_score(y_test, model.fit(X_train, y_train).predict(X_test))

0.8537323821926551

In [635]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Previsões
y_pred = model.predict(X_test)

# Cálculo das métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Exibição dos resultados
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R²: {r2}')

MAE: 286.02078091251326
MSE: 195727.4101211783
RMSE: 442.41090642204824
R²: 0.8537323821926551


In [636]:
base_model.sort_values(by=['score'], ascending=False).head(10)

,aging_due_date,kind_goods,kind_services,active,canceled,finished,not_canceled,installment median,total,score
255,0.0,539,0,471.0,0.0,68.0,539.0,2.0,539.0,9918.448347
77,0.0,808,0,790.0,0.0,18.0,808.0,4.0,808.0,8000.000000
372,0.0,512,0,500.0,0.0,12.0,512.0,3.0,512.0,7000.000000
534,0.0,142,0,115.0,0.0,27.0,142.0,2.0,142.0,6958.591005
84,0.0,192,0,180.0,0.0,12.0,192.0,3.0,192.0,5547.180704
24,0.0,188,0,176.0,0.0,12.0,188.0,1.0,188.0,5471.019799
25,0.0,274,0,257.0,6.0,11.0,268.0,1.0,274.0,5333.469856
526,0.0,636,0,636.0,0.0,0.0,636.0,1.0,636.0,5000.000000
232,0.0,513,0,513.0,0.0,0.0,513.0,4.0,513.0,5000.000000
321,0.0,662,0,662.0,0.0,0.0,662.0,1.0,662.0,5000.000000


In [637]:
# Try to predict score
aging_due_date = 0
kind_goods = 539
kind_services	= 0
active = 471
canceled = 0
finished	=68
not_canceled	= 539
installment_median	= 0
total	= 539

new_test = [aging_due_date,	kind_goods,	kind_services,	active,	canceled,	finished,	not_canceled,installment_median,	total	]

X = np.array(new_test).reshape(1,-1)
X = sc.transform(X)
print("Score do endossante:", model.predict(X))

Score do endossante: [12794.05075326]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [638]:
pesos = model.coef_
intercepto = model.intercept_
print('aging_due_date	kind_goods	kind_services	active		not_canceled	total')
print( pesos)
print("Intercepto:", intercepto)

aging_due_date	kind_goods	kind_services	active		not_canceled	total
[-265.77308374 2337.02317134  389.10793249 2021.28235108 -911.08718727
 6522.21521836 2525.15308441  314.39149276 2390.54653699]
Intercepto: 248.9971503251037
